In [1]:
import pandas as pd
import numpy as np
import random
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.naive_bayes import ComplementNB
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import plot_confusion_matrix
# from sklearn.metrics import classification_report
# import operator
import re
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [2]:
file = "data/movie_reviews.csv"

In [3]:
df = pd.read_csv(file)

In [4]:
df.head()

,Summary,Sentiment
0,rock destined st century new conan going make ...,1
1,gorgeously elaborate continuation lord ring tr...,1
2,effective tepid biopic,1
3,sometimes like go movie fun wasabi good place ...,1
4,emerges something rare issue movie honest keen...,1


In [5]:
df.dropna()

,Summary,Sentiment
0,rock destined st century new conan going make ...,1
1,gorgeously elaborate continuation lord ring tr...,1
2,effective tepid biopic,1
3,sometimes like go movie fun wasabi good place ...,1
4,emerges something rare issue movie honest keen...,1
...,...,...
10657,terrible movie people nevertheless find moving,0
10658,many definition time waster movie must surely one,0
10659,stand crocodile hunter hurried badly cobbled l...,0
10660,thing look like made home video quickie,0


In [6]:
df.drop_duplicates(keep = 'last', inplace = True)

In [7]:
#df.describe()

In [8]:
#df.info()

In [9]:
df["Sentiment"].value_counts()

1    5330
0    5330
Name: Sentiment, dtype: int64

In [10]:
# Assign X (data) and y (target)
X = df.drop("Sentiment", axis=1)
y = df["Sentiment"]
print(X.shape, y.shape)

(10660, 1) (10660,)


In [11]:
# n = 0
# for row in X.iterrows():
#     if n >= 2:
#         break
#     print(row)
#     n += 1

In [12]:
doc_list = []
words_bag = {}

#build documents with each review's words and sentiment in tuple and append into doc_list
for i in X.index:
#     if i >= 1000:
#         break
    sentence = X["Summary"][i]
    res = re.findall(r'\w+', sentence)
    res_tuple = (res,)
    sent_tuple = (y[i],)
    words_sent_tuple = res_tuple + sent_tuple
    doc_list.append(words_sent_tuple)
    
#get the unique words into words_bag
    for j in range(len(res)):
        if res[j] in words_bag:
            words_bag[res[j]] += 1
        else:
            words_bag[res[j]] = 1
#print(words_bag)

In [13]:
# get the whole words in list
words_features = list(words_bag)
print(len(words_features))

16319


In [14]:
#shuffle the doc_list
random.shuffle(doc_list)

In [18]:
def document_features(d_words):
    document_words = set(d_words)
    features = {}
    for word in words_features:
        features['contains({})'.format(word)] = (word in document_words) #if in return "True" otherwise "False".
    return features

In [39]:
# if wanted can working on smaller docs size
working_docs = doc_list[:5000]

In [40]:
#working on smaller sized docs
#featuresets = [(document_features(d), c) for (d,c) in working_docs]

In [16]:
# Train Naive Bayes classifier
featuresets = [(document_features(d), c) for (d,c) in doc_list]
train_set, test_set = featuresets[3200:], featuresets[:3200] #80% train 20% test, total 16319
classifier = nltk.NaiveBayesClassifier.train(train_set)

## Accuracy assessment of the classifier trained

In [19]:
# Test the classifier
print(nltk.classify.accuracy(classifier, test_set))

0.755


## Error Analysis

In [41]:
#working on smaller sized docs
featuresets2 = [(document_features(d), c) for (d,c) in working_docs]

In [48]:
#original divided into 3 groups,train set, test set, and development set
train_set2 = featuresets2[1500:]
devtest_set = featuresets2[500:1500]
test_set2 = featuresets2[:500]

In [49]:
#train classifier2 with smaller size docs
classifier2 = nltk.NaiveBayesClassifier.train(train_set2)

In [53]:
# Test the classifier2
print(nltk.classify.accuracy(classifier2, test_set2))

0.756


In [50]:
errors = []
rights = []
for (d, c) in devtest_set:
    guess = classifier2.classify(document_features(d))
    if guess != c:
        errors.append( (c, guess) ) #tuple as element appended
    else:
        rights.append((c, guess))

In [54]:
len(errors)

510

In [55]:
len(rights)

490

In [38]:
#errors

## save and load the trained classifiers

In [32]:
import pickle
# save the model to disk
filename = 'bayesclass_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [56]:
filename2 = 'bayesclass_model2_smallsize.sav'
pickle.dump(classifier2, open(filename2, 'wb'))

In [57]:
# load the model from disk
classifier = pickle.load(open(filename, 'rb'))
classifier2 = pickle.load(open(filename2, 'rb'))